# Understand embeddings with Word2Vec

### Exercise objectives:
- Convert 🔠 words to 🔢 vector representations thanks to embeddings
- Discover the powerful Word2Vec algorithm

<hr>

_Embeddings_ are representations of words using vectors. These embeddings can be learned within a Neural Network. But it can take time to converge. Another option is to learn them as a first step. Then, use them directly to feed the word representations into a Recurrent Neural Network. 

▶️ Run this cell and make sure the version of 📚 [Gensim - Word2Vec](https://radimrehurek.com/gensim/auto_examples/index.html) you are using is ≥ 4.0!

In [1]:
!pip freeze | grep gensim


gensim==4.2.0


In [2]:
!pip freeze | grep tensorflow


tensorflow-datasets==4.6.0
tensorflow-estimator==2.10.0
tensorflow-macos==2.10.0
tensorflow-metadata==1.10.0


# The data

Keras provides many datasets, among which is the IMDB dataset 🎬:
- It is comprised of sentences that are ***movie reviews***. 
- Each of these reviews is related to a score given by the reviewer.

❓ **Question** ❓ First of all, let's load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer can handle it. Otherwise, rerun with a lower number.  

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy. 

In [3]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)

    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)

        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]

        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]

    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]

    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)


2023-11-23 11:08:46.918951: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQXIOJG/imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQXIOJG/imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


2023-11-23 11:09:14.333619: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


<b><u>Embeddings in the previous challenge</u></b>:

In the previous exercise, we jointly learned a representation for the words, and fed this representation to a RNN, as shown down below 👇: 

<img src="layers_embedding.png" width="400px" />

However, this increases the number of parameters to learn, which slows down and increases the difficulty of convergence!

<b><u>Embeddings in the current challenge</u></b>:

For this reason, we will separate the steps of learning the word representation and feeding it into a RNN. As shown here: 

<img src="word2vec_representation.png" width="400px" />

We will learn the embedding with Word2Vec.

The drawback is indeed that the learned embeddings are not _specifically_ designed for our task. However, learning them independently of the task at hand (sentiment analysis) has some advantages: 
- it is very fast to do in general (with Word2Vec)
- the representation learned by Word2Vec is still meaningful 
- the convergence of the RNN alone will be easier and faster

So let's learn an embedding with Word2Vec and see how meaningful it is!

# Embedding with Word2Vec

Let's use Word2Vec to embed the words of our sentences. Word2Vec will be able to convert each word to a fixed-size vectorial representation.

For instance, we will have:
- 🐶 _dog_ $\rightarrow$ [0.1, -0.3, 0.8]
- 🐱 _cat_ $\rightarrow$ [-1.1, 2.3, 0.7]
- 🍏 _apple_ $\rightarrow$ [3.1, 0.9, -4.7]

Here, your embedding space is of size 3.

***What is a "good" numerical representation of words?***

- ***Words with close meanings should be geometrically close in your embedding space!***

    - Look at the following example which represents a bi-dimensional embedding space.

![Embedding](word_embedding.png)

❓ **Question** ❓ Let's run Word2Vec! 

[📚 **Gensim**](https://radimrehurek.com/gensim/)  is a great Python package that makes the use of the Word2Vec algorithm easy to implement, fast and accurate (which is not an easy task!).

1. The following code imports Word2Vec from Gensim. 

2. The second line learns the embedding representation of the words thanks to the sentences in `X_train`. 
3. The third line stores the words and their trained embeddings in `wv`.

In [4]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train)
wv = word2vec.wv


Let's look at the embedded representation of some words.

You can use `wv` as a dictionary.
For instance, `wv['dog']` will return a representation of `dog` in the embedding space.

❓ **Question** ❓ Try different words - especially, try non-existing words to see that they don't have any representation (which is perfectly normal as their representation was not learned). 

In [5]:
vector_dog = wv['dog']
print("vector for 'dog'", vector_dog)

similar_words_cat = wv.most_similar('cat', topn=5)
print("most similar words to 'cat'", similar_words_cat)

is_apple_present = 'apple' in wv
print("'apple' in vocab", is_apple_present)

is_non_existing_word_present = 'xyz' in wv
print("'xyz' in vocab", is_non_existing_word_present)


vector for 'dog' [-0.18757688  0.17214037 -0.1121115   0.24613217 -0.04401321 -0.41170317
 -0.00150945  0.5882969  -0.22738764 -0.18487713 -0.02495031 -0.3622422
 -0.0280234   0.1228921  -0.03867351 -0.2777799   0.16058654 -0.27053624
 -0.03413136 -0.3616724   0.17426044  0.10248363  0.27523404 -0.09459525
 -0.01315555  0.05532716 -0.17419718 -0.09252553 -0.16163021  0.04336675
  0.15841213  0.02970891  0.17638345 -0.38579282 -0.18088551  0.2683883
  0.09644099 -0.13483062 -0.26809436 -0.3182154  -0.1462763  -0.2750825
 -0.24734311  0.20715292  0.2644375  -0.07959655 -0.11204612 -0.02802587
  0.2309427   0.17947456  0.0892203  -0.17634554 -0.23175058 -0.02709211
 -0.17651504  0.19944666  0.15131396 -0.03327561 -0.21923903  0.03988692
  0.12709989 -0.02808721  0.06007756  0.12126347 -0.2468872   0.28403118
 -0.02596058  0.27217385 -0.32738796  0.26754022 -0.22631681  0.23677689
  0.3525362  -0.23525774  0.3329574   0.16940795 -0.00949523  0.10710374
 -0.18269584 -0.06445752 -0.2956764  

❓ **Question** ❓ What is the size of each word representation, and therefore, what is the size of the embedding space?

In [6]:
word2vec = Word2Vec(sentences=X_train)


🧐 How do we know whether this embedding make any sense or not? 

💡 To investigate this question, we will check that words with a close meaning have close representations. 

👉 Let's use the [**`Word2Vec.wv.most_similar`**](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.most_similar) method that, given an input word, displays the "closest" words in the embedding space. If the embedding is well done, then words with similar meanings will have similar representation in the embedding space.

❓ **Question** ❓ Try out the `most_similar` method on different words. 

🧑🏿‍🏫 The quality of the closeness will depend on the quality of your embedding, and thus, depend on the number of sentences that you have loaded and from which you create your embedding.

In [8]:
similar_words_happy = wv.most_similar('happy', topn=5)
print("most similar words to 'happy'", similar_words_happy)

similar_words_movie = wv.most_similar('movie', topn=5)
print("most similar words to 'movie'", similar_words_movie)

similar_words_good = wv.most_similar('good', topn=5)
print("most similar words to 'good'", similar_words_good)


most similar words to 'happy' [('shock', 0.95807945728302), ('mistake', 0.957321286201477), ('afraid', 0.9565247297286987), ('difficult', 0.9551759362220764), ('forgotten', 0.954578697681427)]
most similar words to 'movie' [('film', 0.9681926965713501), ('show', 0.8605576157569885), ('thing', 0.8370571732521057), ('sequel', 0.8231511116027832), ('series', 0.7961447238922119)]
most similar words to 'good' [('great', 0.9226853847503662), ('funny', 0.8962520956993103), ('bad', 0.8866976499557495), ('quite', 0.8800169229507446), ('very', 0.8096610903739929)]


📚 Similarly to `most_similar` used on words directly, we can use [**`similar_by_vector`**](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.similar_by_vector) on vectors to do the same thing:

In [9]:
vector_happy = wv['happy']
similar_words_by_vector_happy = wv.similar_by_vector(vector_happy, topn=5)
print("most similar words to 'happy' by vector", similar_words_by_vector_happy)


most similar words to 'happy' by vector [('happy', 1.0), ('shock', 0.9580795168876648), ('mistake', 0.957321286201477), ('afraid', 0.9565247297286987), ('difficult', 0.9551759362220764)]


# Arithmetic on words

Now, let's perform some mathematical operations on words, i.e. on their vector representations!

As any learned word is represented as a vector, you can do basic arithmetic operations, such as:

$$W2V(good) - W2V(bad)$$

❓ **Question** ❓ Do this mathematical operation and print the result

In [10]:
vector_dog = wv['dog']
vector_cat = wv['cat']

vector_sum = vector_dog + vector_cat
print("result of adding 'dog' and 'cat' vectors", vector_sum)

vector_diff = vector_dog - vector_cat
print("result of subtracting 'cat' from 'dog' vectors", vector_diff)


result of adding 'dog' and 'cat' vectors [-0.3780874   0.40233445 -0.3598295   0.64811176 -0.08829349 -0.7011212
  0.1265588   1.2927476  -0.469553   -0.39863822 -0.01886719 -0.788615
  0.0875569   0.29772076 -0.07933551 -0.61144435  0.38818616 -0.51949567
 -0.07261017 -0.8496297   0.3688168   0.19793966  0.5520805  -0.32220888
 -0.04255875  0.09987916 -0.4604445  -0.18527298 -0.43691516  0.03159631
  0.33770233  0.01401589  0.34468293 -0.95224524 -0.46717703  0.5089961
  0.2863616  -0.31260616 -0.64327186 -0.6527047  -0.36637282 -0.6374229
 -0.6859208   0.4530704   0.62930727 -0.11458215 -0.22434145 -0.10539632
  0.59582114  0.36883974  0.18878356 -0.40387225 -0.42680663 -0.10467807
 -0.41618714  0.35286096  0.342733   -0.08266667 -0.36324543  0.13622943
  0.33904266 -0.04693536  0.05102577  0.24800105 -0.37617844  0.52967864
 -0.08483322  0.6424049  -0.7989346   0.6472899  -0.5705408   0.4490307
  0.7648039  -0.5895406   0.69020975  0.288477   -0.00875972  0.34962994
 -0.37449676 -0.

Now, imagine for a second that the following equality holds true:

$$W2V(good) - W2V(bad) = W2V(nice) - W2V(stupid)$$

which is equivalent to:

$$W2V(good) - W2V(bad) + W2V(stupid) = W2V(nice)$$

❓ **Question** ❓ Let's, just for fun (as it would be bold of us to think that this equality holds true ...), do the operation $W2V(good) - W2V(bad) + W2V(stupid)$ and store it in a `res` variable (which will be a vector of size 100 that you can print).

In [11]:
vector_good = wv['good']
vector_bad = wv['bad']
vector_stupid = wv['stupid']

res = vector_good - vector_bad + vector_stupid

print("result of the operation ", res)


result of the operation  [ 0.04952022 -0.40262568  0.26652074  0.02471781  0.18143782 -0.9644551
 -0.34560388  0.72128    -0.6370332  -0.5973649  -0.13724904 -0.5526273
  0.22072993  0.47704712  0.13755813 -0.0115336   0.22345015 -0.43545112
 -0.5658456  -1.2687619   0.5776885   0.2119718   0.70374256 -0.188441
 -0.06637791 -0.3049129  -0.36038163  0.57843095 -0.5208605  -0.6597662
  0.41569728  0.4659888   0.17092709 -0.05024342 -0.2970833  -0.03740638
  0.14699848  0.07255055 -0.47037354  0.19670886 -0.20653987 -0.69768196
 -0.26396775  0.15514898  0.50949913  0.04860783  0.10113779 -0.4816431
 -0.40133846  0.4393894  -0.14696726 -0.11667076 -0.57761073  0.17371173
  0.32254323  0.14384514 -0.18140045  0.33101138 -0.23917913  0.52751195
  0.11038022  0.08554247  0.0225382  -0.2000148  -0.28912252  0.40271848
 -0.47041333 -0.19116127  0.14172551  0.69181263 -0.59934413  0.0289579
  0.28979242  0.31640422  0.43028486 -0.08830927  0.11124785 -0.21164057
 -0.10057187  0.08025627  0.11180

We said earlier, that for any vector it is possible to see the closest vectors in the embedding space.

❓ **Question** ❓ Look at the closest vectors of `res`

💡 _Hint_: `similar_by_vector`

In [12]:
similar_words_res = wv.similar_by_vector(res, topn=5)
print("most similar words to 'res' vector", similar_words_res)


most similar words to 'res' vector [('nice', 0.7579593658447266), ('potential', 0.7270023822784424), ('good', 0.7231265902519226), ('handful', 0.7207500338554382), ('decent', 0.7121587991714478)]


Incredible right! You can do arithmetic operations on words!

❓ **Question** ❓ You can try on arithmetic such as 

$$W2V(Boy) - W2V(Girl) = W2V(Man) - W2V(Woman)$$

or 

$$W2V(Queen) - W2V(King) = W2V(actress) - W2V(actor)$$

❗ **Remark** ❗ You will probably see that the results are not perfect. But don't forget that you trained your model on a very small corpus.

In [13]:
vector_boy = wv['boy']
vector_girl = wv['girl']
vector_man = wv['man']
vector_woman = wv['woman']

result_gender = vector_boy - vector_girl - (vector_man - vector_woman)

similar_words_gender = wv.similar_by_vector(result_gender, topn=5)
print("most similar words to 'result_gender' vector", similar_words_gender)


most similar words to 'result_gender' vector [('10', 0.585750162601471), ('1', 0.532700777053833), ('3', 0.48187926411628723), ('2', 0.4649432599544525), ('8', 0.4625183641910553)]


<u><i>Some notes about Word2Vec as an internal Neural Network</i></u>:

You might wonder where does this magic comes from (at quite a low price, you just ran a line of code on a very small corpus and it was trained within few minutes). The magic comes from the way Word2Vec is trained. The details are quite complex, but you can remember that Word2vec, in `word2vec = Word2Vec(sentences=X_train)`, actually trains a internal neural network (that you don't see).  

In a nutshell, this internal neural network predicts a word from the surroundings words in a sentences. Hence, it splits the original sentences, then for each split it chooses some words as inputs $X$ and a word as the output $y$ which it tries to predict, using the embedding space.

And as with any neural network, Word2Vec has some hyperparameters. Let's play with some of these. 

# Word2Vec hyperparameters

❓ **Question** ❓ The first important hyperparameter is the `vector_size` argument. It corresponds to the size of the embedding space. Learn a new `word2vec_2` model, still trained on the `X_train`, but with a smaller or higher `vector_size`.

Verify on some words that the embedding size is the one you chose.

In [14]:
word2vec_2 = Word2Vec(sentences=X_train, vector_size=50)
wv_2 = word2vec_2.wv

vector_cat_2 = wv_2['cat']
vector_dog_2 = wv_2['dog']

print("vector size for 'cat' in word2vec_2", vector_cat_2.shape)
print("vector size for 'dog' in word2vec_2", vector_dog_2.shape)


vector size for 'cat' in word2vec_2 (50,)
vector size for 'dog' in word2vec_2 (50,)


❓ **Question** ❓ Use the **`Word2Vec.wv.key_to_index`** attribute to display the size of the learned vocabulary. Compare it to the number of different words in `X_train`.

In [15]:
vocab_size_word2vec_2 = len(wv_2.key_to_index)
print("size of the learned vocabulary in word2vec_2", vocab_size_word2vec_2)

unique_words_in_X_train = len(set(word for sentence in X_train for word in sentence))
print("num of different words in X_train", unique_words_in_X_train)


size of the learned vocabulary in word2vec_2 8006
num of different words in X_train 30419


There is an important difference between the number of words in the train sentences and in the Word2Vec vocabulary, even though it has been trained on the train sentence set. The reasons comes from the second important hyperparameter of Word2Vec:  `min_count`. 

`min_count` is a integer that tells you how many occurrences a given word should have to be learned in the embedding space. For instance, let's say that the word "movie" appears 1000 times in the corpus and "simba" only 2 times. If `min_count=3`, the word "simba" will be skipped during the training.

The intention is to learn a representation of words that are sufficiently present in the corpus to have a robust embedded representation.

❓ **Question** ❓ Learn a new `word2vec_3` model with a `min_count` higher than 5 (which is the default value) and a `word2vec_4` with a `min_count` smaller than 5, and then, compare the size of the vocabulary for all the different word2vecs that you have trained (you can choose any `vector_size` you want).

In [16]:
word2vec_3 = Word2Vec(sentences=X_train, vector_size=50, min_count=10)
wv_3 = word2vec_3.wv

word2vec_4 = Word2Vec(sentences=X_train, vector_size=50, min_count=2)
wv_4 = word2vec_4.wv

vocab_size_word2vec_2 = len(wv_2.key_to_index)
vocab_size_word2vec_3 = len(wv_3.key_to_index)
vocab_size_word2vec_4 = len(wv_4.key_to_index)

print("size of the vocabulary in word2vec_2", vocab_size_word2vec_2)
print("size of the vocabulary in word2vec_3", vocab_size_word2vec_3)
print("size of the vocabulary in word2vec_4", vocab_size_word2vec_4)


size of the vocabulary in word2vec_2 8006
size of the vocabulary in word2vec_3 4503
size of the vocabulary in word2vec_4 16729


Remember that Word2Vec has an internal neural network that is optimized based on some predictions. These predictions actually correspond to predicting a word based on surrounding words. The surroundings words are in a `window` which corresponds to the number of words taken into account. And you can train the Word2Vec with different `window` sizes.

❓ **Question** ❓ Train a new `word2vec_5` model with a `window` different than previously (default is 5).

In [17]:
word2vec_5 = Word2Vec(sentences=X_train, vector_size=50, window=10)
wv_5 = word2vec_5.wv


The arguments you have seen (`vector_size`, `min_count` and `window`) are usually the ones that you should start playing with to get a better performance for your model.

But you can also look at other arguments in the [**📚 Documentation - gensim.models.word2vec.Text8Corpus**](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Text8Corpus)

# Convert our train and test set to RNN-ready datasets

Remember that `Word2Vec` is the first step to the overall process of feeding such a representation into a RNN, as shown here:

<img src="word2vec_representation.png" width="400px" />



Now, let's work on Step 2 by converting the training and test data into their vector representation to be ready to be fed in RNNs.

❓ **Question** ❓ Now, write a function that, given a sentence, returns a matrix that corresponds to the embedding of the full sentence, which means that you have to embed each word one after the other and concatenate the result to output a 2D matrix (make sure that your output is a NumPy array)

❗ **Remark** ❗ You will probably notice that some words you are trying to convert throw errors as they are said not to belong to the dictionary:

- For the <font color=orange>test</font> set, this is understandable: <font color=orange>some words were not</font> in the <font color=blue>train</font> set and thus, their <font color=orange>embedded representation is unknown</font>
- for the <font color=blue>train set</font>, due to `min_count` hyperparameter, not all the words have a vector representation.

In any case, just skip the missing words here.

In [19]:
import numpy as np

def embed_sentence(word2vec, sentence):
    word_vectors = []

    for word in sentence:
        if word in word2vec:
            word_vectors.append(word2vec[word])

    if len(word_vectors) == 0:
        return np.zeros((0, word2vec.vector_size))

    mean_vector = np.mean(word_vectors, axis=0)

    return mean_vector


❓ **Question** ❓ Write a function that, given a list of sentences (each sentence being a list of words/strings), returns a list of embedded sentences (each sentence is a matrix). Apply this function to the train and test sentences

💡 _Hint_: Use the previous function `embed_sentence`

In [20]:
import numpy as np

def embedding(word2vec, sentences):
    embedded_sentences = []

    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)

        embedded_sentences.append(embedded_sentence)

    return embedded_sentences


❓ **Question** ❓ In order to have ready-to-use data, do not forget to pad your sequences so you have tensors which can be divided into batches (of `batch_size`) during the optimization. Store the padded values in `X_train_pad` and `X_test_pad`. Do not forget the important arguments of the padding ;)

In [21]:
def embedding(word2vec, sentences):
    embedded_sentences = []
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embedded_sentences.append(embedded_sentence)
    return embedded_sentences




🏁 Congratulations, you are now able to use `Word2Vec` to embed your words :)

💾 Don't forget to git add/commit/push your notebook...

🚀 ... and move on to the next challenge!
